In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from itertools import islice

from torch import nn
from torch.nn import functional as F

import sys 
import getpass

sys.path.append('../Proj_308427_348143_XXXXXX')
from Miniproject_2.model import *
torch.set_grad_enabled(True);

In [2]:
train_input, train_target = torch.load('../train_data.pkl')

select=100
x     = (train_input[:select].float()/255.).requires_grad_()
xtrue = (train_target[:select].float()/255.)

In [3]:
f = torch.empty(5,3,3,3)

f[0, 0] = torch.tensor([ [ +0., +0. ,1 ], [ +0., +1., 1], [1,0,1]])
f[1, 0] = torch.tensor([ [ +1., +0. ,1 ], [ +0., +1., 1], [1,0,1]])
f[2, 0] = torch.tensor([ [ -1., +0. ,1 ], [ -1., +0., 1], [1,0,1]])
f[3, 0] = torch.tensor([ [ -1., -1. ,1 ], [ +1., +0., 1], [1,0,1]])
f[4, 0] = torch.tensor([ [ +0., -1. ,1 ], [ -1., +2., 1], [1,0,1]])

for j in range(0,5):
    for i in range(1,3):
        f[j,i] = f[j,0]
        
f.requires_grad_();

In [4]:
stride = 3
y = F.conv2d(x, f, stride=stride)

with torch.no_grad():
    ytrue = F.conv2d(xtrue, f, stride=stride)

L = F.mse_loss(y,ytrue)
dL_dy, dL_dx, dL_df = torch.autograd.grad(L, (y,x,f))

print("Shapes")
print("x : \t", tuple(x.shape))
print("y : \t", tuple(y.shape))
print("f : \t", tuple(f.shape))
print("dL_dy : ", tuple(dL_dy.shape))
print("dL_dx : ", tuple(dL_dx.shape))
print("dL_df : ", tuple(dL_df.shape))

Shapes
x : 	 (100, 3, 32, 32)
y : 	 (100, 5, 10, 10)
f : 	 (5, 3, 3, 3)
dL_dy :  (100, 5, 10, 10)
dL_dx :  (100, 3, 32, 32)
dL_df :  (5, 3, 3, 3)


In [5]:
mydL_dx, mydL_df = conv_backward(x, dL_dy, f, stride=stride)

In [7]:
print(torch.allclose(mydL_df, dL_df, rtol=1e-5))
print(torch.allclose(mydL_dx, dL_dx, rtol=1e-5))

True
True


In [8]:
valid_input, valid_target = torch.load('../val_data.pkl')

select=10
x     = (valid_input[:select].float()).requires_grad_()
xtrue = (valid_target[:select].float())

In [14]:
f = torch.empty(5,3,3,3)

f[0, 0] = torch.tensor([ [ +0., +0. ,1 ], [ +0., +1., 1], [1,0,1]])
f[1, 0] = torch.tensor([ [ +1., +0. ,1 ], [ +0., +1., 1], [1,0,1]])
f[2, 0] = torch.tensor([ [ -1., +0. ,1 ], [ -1., +0., 1], [1,0,1]])
f[3, 0] = torch.tensor([ [ -1., -1. ,1 ], [ +1., +0., 1], [1,0,1]])
f[4, 0] = torch.tensor([ [ +0., -1. ,1 ], [ -1., +2., 1], [1,0,1]])

for j in range(0,5):
    for i in range(1,3):
        f[j,i] = f[j,0]
        
ff = f.transpose(0,1)
ff.requires_grad_();

In [15]:
stride = 3
y = F.conv_transpose2d(x, ff, stride=stride)

with torch.no_grad():
    ytrue = F.conv_transpose2d(xtrue, ff, stride=stride)

L = F.mse_loss(y,ytrue)
dL_dy, dL_dx, dL_df = torch.autograd.grad(L, (y,x,ff))

print("Shapes")
print("x : \t", tuple(x.shape))
print("y : \t", tuple(y.shape))
print("f : \t", tuple(f.shape))
print("dL_dy : ", tuple(dL_dy.shape))
print("dL_dx : ", tuple(dL_dx.shape))
print("dL_df : ", tuple(dL_df.shape))

Shapes
x : 	 (10, 3, 32, 32)
y : 	 (10, 5, 96, 96)
f : 	 (5, 3, 3, 3)
dL_dy :  (10, 5, 96, 96)
dL_dx :  (10, 3, 32, 32)
dL_df :  (3, 5, 3, 3)


In [16]:
mytconv = TransposeConv2d(3 ,5, 3, stride=stride, padding=0, dilation=1)
mytconv.weight = ff

In [17]:
_, vjp = mytconv.forward_and_vjp(x.detach())

mydL_dx, mydL_df, _ = vjp(dL_dy)

print("Shapes")
print("mydL_dx : ",mydL_dx.shape)
print("mydL_df : ",mydL_df.shape)

Shapes
mydL_dx :  torch.Size([10, 3, 32, 32])
mydL_df :  torch.Size([3, 5, 3, 3])


In [18]:
print(torch.allclose(mydL_df, dL_df, rtol=1e-5))
print(torch.allclose(mydL_dx, dL_dx, rtol=1e-5))

True
True
